## Trajectories for m-miba

In [85]:

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go


from scipy.spatial.transform import Rotation as R

from typing import Tuple, List

# After installing ipympl and restarting backend
%matplotlib widget

## Generate Contact Points

In [156]:
#parameters
OFFSET_X = 0
OFFSET_Y = 0
OFFSET_Z = 1

RANGE_X = [-0,0]
RANGE_Y = [-0,0]
RANGE_Z = [0,1]


NUM_SAMPLES_X = 1
NUM_SAMPLES_Y = 1
NUM_SAMPLES_Z = 1

X_PLOTTING_RANGE = [-17/2.0,17/2.0]
Y_PLOTTING_RANGE = [-71/2.0,71/2.0]
Z_PLOTTING_RANGE = [-4,1]


In [157]:
# sample_random_points()


In [158]:

def sample_ordered_points(ranges=[[0, 1], [0, 1], [0, 1]], 
                            num_samples=[10, 10, 10], 
                            offsets=[0, 0, 0]):
    """
    Generates 3D points within the specified x, y, z ranges.

    Args:
        ranges (list): List of ranges for x, y, z axes.
                       Format: [[x_start, x_end], [y_start, y_end], [z_start, z_end]]
        num_samples (list): List of the number of samples for x, y, z axes.
                            Format: [num_samples_x, num_samples_y, num_samples_z]
        offsets (list): List of offsets to apply to each axis [x_offset, y_offset, z_offset].

    Returns:
        np.array: Array of shape (num_samples_x * num_samples_y * num_samples_z, 3)
                  containing (x, y, z) points.
    """
    # Unpack range and sample settings for x, y, z
    range_x, range_y, range_z = ranges
    num_x, num_y, num_z = num_samples
    offset_x, offset_y, offset_z = offsets

    # Generate points for each axis, including offsets
    x_points = np.linspace(range_x[0], range_x[1], num_x) + offset_x
    y_points = np.linspace(range_y[0], range_y[1], num_y) + offset_y
    z_points = np.linspace(range_z[0], range_z[1], num_z) + offset_z

    # Create meshgrid for x, y, and z and reshape to a list of (x, y, z) points
    x_grid, y_grid, z_grid = np.meshgrid(x_points, y_points, z_points, indexing='ij')
    p_sensor_contactpt = np.column_stack((x_grid.ravel(), y_grid.ravel(), z_grid.ravel()))

    ## Map for m-miba mount (z pointing down) - corrects the orientation 
    # x -> y
    # y -> x
    # z -> -z
    T_zMount_sensor = np.array([[0, 1, 0], [1, 0, 0], [0, 0, -1]])
    p_zMount_contactpt = (T_zMount_sensor @ p_sensor_contactpt.T).T
    
    return p_zMount_contactpt



In [159]:
def plot_points(points, plotting_ranges):
    """
    Plots the contact points in 2D (x, y) and 3D (x, y, z) with specified axis ranges.
    
    Args:
        points (np.array): Array of shape (N, 3) with (x, y, z) points.
        x_range (list, optional): Range for x-axis [xmin, xmax].
        y_range (list, optional): Range for y-axis [ymin, ymax].
        z_range (list, optional): Range for z-axis [zmin, zmax] (for 3D plot).
    """

    x_plotting_range, y_plotting_range, z_plotting_range = plotting_ranges
    # 2D plot (x, y)
    fig_2d = px.scatter(x=points[:, 0], y=points[:, 1], title='2D Plot of (x, y) Points')
    fig_2d.update_xaxes(title='x', range=x_plotting_range, scaleanchor="y", scaleratio=1)
    fig_2d.update_yaxes(title='y', range=y_plotting_range, scaleanchor="x", scaleratio=1)
    fig_2d.show()


    
    # 3D plot (x, y, z)
    aspect_ratio = dict(x=1, y=1, z=1)

    fig_3d = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(size=3, color=points[:, 2], colorscale='Viridis', opacity=0.8)
    )])
    fig_3d.update_layout(scene=dict(
        xaxis=dict(title='x', range=x_plotting_range),
        yaxis=dict(title='y', range=y_plotting_range),
        zaxis=dict(title='z', range=z_plotting_range),
        aspectmode="manual", aspectratio=aspect_ratio),
        title='3D Plot of (x, y, z) Points')
    fig_3d.show()


In [160]:
points = sample_ordered_points(ranges = [RANGE_X, RANGE_Y, RANGE_Z], 
                          num_samples = [NUM_SAMPLES_X, NUM_SAMPLES_Y, NUM_SAMPLES_Z], 
                          offsets = [OFFSET_X, OFFSET_Y, OFFSET_Z])
plot_points(points, plotting_ranges = [X_PLOTTING_RANGE, Y_PLOTTING_RANGE, Z_PLOTTING_RANGE])

## Create A Full Trajectory
We take the points generated and convert them to a continuous trajectory of points comprising of:

1. the point to be sampled 
2. the location to retract to after sampling a point
3. the location to start the the approach to for the the next sample point (this is the same as the point to retract to)

The I_tare flag is added to the approach points to indicate when to tare the collected data. 

The t_dwell time indicates how long to stay at the given point.

The final trajectory is a an array of N points to traverse where each row is a list of 7 values:

$$(x, y, z, \theta, \phi, T_{dwell}, \mathbb{I}_{contact})


In [161]:

def convert_mountPoints_to_robot_coordinates(points):
    #

    # Number of rows (points) in p_gantry_contactPoint
    N = points.shape[0]

    # Create theta and phi columns filled with zeros
    theta = np.zeros((N, 1))  # Column vector of zeros with shape (N, 1)
    phi = np.zeros((N, 1))    # Column vector of zeros with shape (N, 1)
    raw_commands = np.hstack((points, theta, phi))
    return raw_commands


def make_approach_points(points, approach_offset_from_center):
    N = points.shape[0]
    z_approach_points = -approach_offset_from_center*np.ones((N,1))
    approach_points = np.hstack((points[:,0:1],points[:,1:2], z_approach_points))
    return approach_points

def broadcast_t_dwell(coordinate, t_dwell):
    return np.hstack((coordinate, np.ones((coordinate.shape[0], 1)) * t_dwell))

def broadcast_I_tare(coordinate, I_tare):
    return np.hstack((coordinate, np.ones((coordinate.shape[0], 1)) * I_tare))

def convert_points_to_trajectory(points):

    approach_offset_from_center = 10  # Must be greater than traversal height
    
    mmiba_coordinates = []
    coordinate_length = 7 #[x, y, z, theta, phi, t_dwell, I_tare_flag]
    
    p_mount_approach = make_approach_points(points, approach_offset_from_center)
    home_coordinate = [0,0,-approach_offset_from_center, 0, 0, 0, 0]

    contact_coordinates = convert_mountPoints_to_robot_coordinates(points)
    approach_coordinates = convert_mountPoints_to_robot_coordinates(p_mount_approach)
    retract_cooridnates = convert_mountPoints_to_robot_coordinates(p_mount_approach)

    # Time to sample = 1s
    t_dwell_contact = 1

    contact_coordinates = broadcast_t_dwell(contact_coordinates, t_dwell=t_dwell_contact)
    approach_coordinates = broadcast_t_dwell(approach_coordinates, 0.5)
    retract_cooridnates = broadcast_t_dwell(retract_cooridnates, 0.5) 

    contact_coordinates = broadcast_I_tare(contact_coordinates, 0)
    approach_coordinates = broadcast_I_tare(approach_coordinates, 1)
    retract_cooridnates = broadcast_I_tare(retract_cooridnates, 0)

    merged_coordinates = np.zeros((contact_coordinates.shape[0] * 3, coordinate_length))
    merged_coordinates[::3] = approach_coordinates
    merged_coordinates[1::3] = contact_coordinates
    merged_coordinates[2::3] = approach_coordinates
    

    # Go to home at end
    print(merged_coordinates.shape)
    merged_coordinates = np.vstack((merged_coordinates, home_coordinate))
    print(merged_coordinates.shape)



    mmiba_coordinates.append(merged_coordinates)
    
    return mmiba_coordinates




In [162]:
points = sample_ordered_points(ranges = [RANGE_X, RANGE_Y, RANGE_Z], 
                          num_samples = [NUM_SAMPLES_X, NUM_SAMPLES_Y, NUM_SAMPLES_Z], 
                          offsets = [OFFSET_X, OFFSET_Y, OFFSET_Z])

mmibaTrajectory = convert_points_to_trajectory(points)


mmibaTrajectory = mmibaTrajectory[0]

idx = 1
save_traj_path = f"trajectories/mmibaTrajectory_{idx}.npy"
np.save(save_traj_path, mmibaTrajectory)

(3, 7)
(4, 7)
